# Reddit 댓글 규칙 위반 여부 분류

## Import Library

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

In [2]:
!kaggle competitions download -c jigsaw-agile-community-rules
!ls -alh
!unzip -l jigsaw-agile-community-rules.zip

Traceback (most recent call last):
  File "/opt/anaconda3/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/opt/anaconda3/lib/python3.12/site-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/opt/anaconda3/lib/python3.12/site-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /Users/iyumin/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
total 48
drwxr-xr-x   5 iyumin  staff   160B 10 13 02:58 .
drwxr-xr-x  16 iyumin  staff   512B 10 12 16:58 ..
drwxr-xr-x   3 iyumin  staff    96B 10 12 17:00 .ipynb_checkpoints
drwxr-xr-x   3 iyumin  staff    96B 10 12 17:08 data
-rw-r--r--   1 iyumin  staff    20K 10 13 02:58 project01.ipynb
unzip:  cannot find or open jigsaw-agile-community-rules.zip, jigsaw-agile-community-rules.zip.zip or jigs

In [3]:
# 새 폴더 만들기 / -p: 중간 경로가 없어도 부모 폴더까지 같이 생성 (이미 있어도 에러 없이 넘어감)
!mkdir -p data/jigsaw_agile

# 압축 풀기 / -q: 조용히 진행, -o(overwrite): 같은 이름의 파일이 있어도 덮어씀
# -d: 풀린 파일을 저장할 목적지 폴더 지정
!unzip -q -o jigsaw-agile-community-rules.zip -d data/jigsaw_agile

# 폴더 내용 보기 / -a: 숨김 파일까지 모두 표시, -l: 자세히 표시, -h: 파일을 KB/MB 단위로 표시
!ls -alh data/jigsaw_agile

unzip:  cannot find or open jigsaw-agile-community-rules.zip, jigsaw-agile-community-rules.zip.zip or jigsaw-agile-community-rules.zip.ZIP.
total 0
drwxr-xr-x  2 iyumin  staff    64B 10 12 17:08 .
drwxr-xr-x  3 iyumin  staff    96B 10 12 17:08 ..


## 1. Load Data

In [4]:
train_df = pd.read_csv('../data/jigsaw_agile/train.csv')
test_df = pd.read_csv('../data/jigsaw_agile/test.csv')

print(test_df.shape)
print(train_df.shape)

(10, 8)
(2029, 9)


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029 entries, 0 to 2028
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   row_id              2029 non-null   int64 
 1   body                2029 non-null   object
 2   rule                2029 non-null   object
 3   subreddit           2029 non-null   object
 4   positive_example_1  2029 non-null   object
 5   positive_example_2  2029 non-null   object
 6   negative_example_1  2029 non-null   object
 7   negative_example_2  2029 non-null   object
 8   rule_violation      2029 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 142.8+ KB


In [6]:
train_df.head()

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0
2,2,Lol. Try appealing the ban and say you won't d...,No legal advice: Do not offer or request legal...,pcmasterrace,Don't break up with him or call the cops. If ...,It'll be dismissed: https://en.wikipedia.org/w...,Where is there a site that still works where y...,Because this statement of his is true. It isn'...,1
3,3,she will come your home open her legs with an...,"No Advertising: Spam, referral links, unsolici...",sex,Selling Tyrande codes for 3€ to paypal. PM. \n...,tight pussy watch for your cock get her at thi...,NSFW(obviously) http://spankbang.com/iy3u/vide...,Good News ::Download WhatsApp 2.16.230 APK for...,1
4,4,code free tyrande --->>> [Imgur](http://i.imgu...,"No Advertising: Spam, referral links, unsolici...",hearthstone,wow!! amazing reminds me of the old days.Well...,seek for lady for sex in around http://p77.pl/...,must be watch movie https://sites.google.com/s...,We're streaming Pokemon Veitnamese Crystal RIG...,1


In [7]:
# 클래스 불균형 확인
train_df["rule_violation"].value_counts()

rule_violation
1    1031
0     998
Name: count, dtype: int64

## 2. Data Preprocessing

In [8]:
import re

def preprocess_text(text):
    # 소문자로 통일
    text = str(text).lower()

    # url이 포함된 경우 [URL] 토큰으로 치환
    text = re.sub(r'http\S+|www\S+', '[URL]', text)

    # 특수문자 제거
    text = re.sub(r'[^\w\s[\]]', '', text)

    # 공백 제거
    text = ' '.join(text.split())
    return text

In [9]:
body_processed = train_df["body"].apply(preprocess_text)
rule_processed = train_df["rule"].apply(preprocess_text)

# 타겟 컬럼 분리
X = body_processed + " [RULE] " + rule_processed
y = train_df["rule_violation"]

In [10]:
# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [11]:
print(X_train)

185     someone needs to dox these bitches real quick ...
1641    i would keep it to yourself if it comes up pre...
1714    look that pussy will and be in on couch make a...
1917    thats when you punch them repeatedly until the...
1433    earn money with android phone just go here [UR...
                              ...                        
467     rapper straight outta cross keys sc youtube se...
1060    from a purely legal standpoint what would be t...
1021    get a lawyer [URL] [RULE] no legal advice do n...
677     if anyone is interested in donating 5 or 10 bu...
1816    [URL] free reddit karma with this glitch fast ...
Length: 1420, dtype: object


## 모델 생성

In [12]:
!pip install transformers

In [13]:
from transformers import pipeline

clf = pipeline("text-classification",
              model="unitary/toxic-bert")

Device set to use mps:0


In [14]:
sample_predictions = []
for text in X_val[:200]:
    result = clf(text, truncation=True)

    # TOXIC로 예측된 경우 1, 아닌 경우 0으로 변환
    label = 1 if result[0]['label'] == 'TOXIC' else 0
    sample_predictions.append(label)

acc = accuracy_score(y_val[:200], sample_predictions)
print(f"Sample Accuracy: {acc}")

Sample Accuracy: 0.51
